In [1]:
import os
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4699592616084322160
]


In [6]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'tensorflow_core._api.v2.config' has no attribute 'experimental_list_devices'

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [2]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [3]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [ ]:
# Builde model
model = Sequential() # Sequential groups a linear stack of layers 
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

model.add(Flatten()) # Flatten the featuremaps
model.add(Dense(units=512)) # Add dense layer with 512 neurons
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# initiate SGD optimizer
opt = keras.optimizers.SGD()

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 10

# Fit the data into model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)




Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 165s 3ms/step - loss: 1.8552 - accuracy: 0.3374 - val_loss: 1.5813 - val_accuracy: 0.4150
Epoch 2/10
50000/50000 [==============================] - 148s 3ms/step - loss: 1.4131 - accuracy: 0.5003 - val_loss: 1.3397 - val_accuracy: 0.5200
Epoch 3/10
50000/50000 [==============================] - 164s 3ms/step - loss: 1.2529 - accuracy: 0.5593 - val_loss: 1.2617 - val_accuracy: 0.5550
Epoch 4/10
50000/50000 [==============================] - 175s 3ms/step - loss: 1.1438 - accuracy: 0.5997 - val_loss: 1.1889 - val_accuracy: 0.5887
Epoch 5/10
50000/50000 [==============================] - 170s 3ms/step - loss: 1.0611 - accuracy: 0.6288 - val_loss: 1.1795 - val_accuracy: 0.5953
Epoch 6/10
50000/50000 [==============================] - 163s 3ms/step - loss: 0.9859 - accuracy: 0.6559 - val_loss: 1.0544 - val_accuracy: 0.6411
Epoch 7/10
50000/50000 [==============================] - 17

In [10]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [11]:
y_pred[0] 

array([2.4284909e-03, 2.9643339e-01, 1.2415329e-03, 3.2413865e-03,
       1.1167271e-03, 1.0127937e-03, 6.8496183e-06, 3.5009726e-03,
       3.1431669e-03, 6.8787467e-01], dtype=float32)

In [12]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

9

In [13]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
y_pred = your_model.predict(x_test)

In [14]:
assert y_pred.shape == (10000,)

In [15]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test-set:  0.6769
